In [23]:
%pip install shutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 45.0/45.0 kB 2.2 MB/s eta 0:00:00
Using legacy 'setup.py install' for shutils, since package 'wheel' is not installed.
  Running setup.py install for shutils: started
  Running setup.py install for shutils: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# %pip install openpyxl

In [2]:
import os
import pytesseract
import pandas as pd
from PIL import Image
import ollama
from datetime import datetime
import warnings
import json
import shutil

warnings.filterwarnings("ignore")

#path to type tesseract executable
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"



#=========configuerations======================================================================================

bill_path='../Bills'
ARCHIVE_FOLDER = '../Archive'
EXCEL_PATH='../Invoices.xlsx'





#=========utility functions======================================================================================

#function to extract text from images in a given folder
def extract_text(image_path):
    if os.path.exists(image_path):
        image=Image.open(image_path)
        text=pytesseract.image_to_string(image,config='--psm 6')
        return text
    else:
        print('Folder does not exists')

#function to generate a prompt

def generate_prompt(text):
        prompt=f""" Youare a helpful assistant that parse the text
            receives from OCR extraction into a JSON format.

            Here is the  that is provided to you.
            {text}

            Your task is to extract the following information: and convert it into a clear json format.
            Make sure to include the following fields only and nothing more:
            Vendor name, Date, Total Amount, Payment Method.

            The output should be a JSON object with the following structure:


            Here is an example for you for the final outtput:

            {{'vendor_name': 'Amazon',
            'date': '2023-10-01',
            'total_amount': 150.75,
            'payment_method': 'Credit Card',
            }}

            Please provide only the JSON reponse and nothing else.

            """
        return prompt

#calling ollama

def call_ollama(prompt):
     response = ollama.chat(
          model='llama3.2',
          messages=[{'role':'user','content':prompt}]
     )
     return response['message']['content']  




# === Create archive path ===
def get_today_archive_path():
    today = datetime.today().strftime("%Y-%m-%d")
    path = os.path.join(ARCHIVE_FOLDER, today)
    os.makedirs(path, exist_ok=True)
    return path



#=========main function======================================================================================
all_data=[]


for file in os.listdir(bill_path):
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.pdf')):
        file_path = os.path.join(bill_path, file)
        print(f"Processing file: {file_path}")

        # Extract text from the image
        text = extract_text(file_path)
        prompt = generate_prompt(text)
        response = call_ollama(prompt)


        try:
            data = json.loads(response)
            all_data.append(data)

            # Move to archive
            archive_path = get_today_archive_path()
            shutil.move(file_path, os.path.join(archive_path, file))
        except json.JSONDecodeError:
            print(f"JSON parse failed for {file}")
            print(response)

        #move to new folder
        # archive_path=get_today_archive_path()
        # shutil.move(file_path,os.path.join(archive_path,file))

# Create a DataFrame from the collected data
if all_data:
    df = pd.DataFrame(all_data)
    df.insert(0, "entry_date", datetime.today().strftime("%Y-%m-%d"))

    if os.path.exists(EXCEL_PATH):
        existing_df = pd.read_excel(EXCEL_PATH)
        df = pd.concat([existing_df, df], ignore_index=True)

    df.to_excel(EXCEL_PATH, index=False)
    print("Data saved to invoices.xlsx")
else:
    print("No valid data extracted.")


print(df)

















No valid data extracted.


NameError: name 'df' is not defined